# 图片集读入并保存到tfrecord文件中

In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import glob
import random

#### 定义转换函数

In [5]:
def images_convert():
    rst = []
    # 数字
    cwd = 'data/'
    for i in range(10):
        num = str(i)
        path = cwd + num + '/*.*'
        for jpgfile in glob.glob(path):
            img = Image.open(jpgfile) #每一个图片的地址
            img = img.resize((24, 48), Image.BILINEAR) #裁剪图片大小
            new_img = img.convert('L') #转为灰度图像
            new_img = np.array(new_img, dtype='uint8')
            new_img = new_img.reshape(48, 24)
            # plt.imshow(new_img, cmap='gray')    
            # plt.show()
            new_img = new_img.reshape(1152,)
            new_img = new_img.tolist()  
            new_img.insert(0, i)
            # 保存
            rst.append(new_img)
    
    # 字母
    for i in range(26):
        letter = chr(65+i)
        path = cwd + letter + '/*.*'
        for jpgfile in glob.glob(path):
            img = Image.open(jpgfile)
            img = img.resize((24, 48), Image.BILINEAR)
            new_img = img.convert('L')
            new_img = np.array(new_img, dtype='uint8')
            new_img = new_img.reshape(48, 24)
            # plt.imshow(new_img, cmap='gray')    
            # plt.show()
            new_img = new_img.reshape(1152,)
            new_img = new_img.tolist()  
            new_img.insert(0, 65+i)
            rst.append(new_img)

    random.shuffle(rst)
    rst = np.array(rst)
    np.reshape(rst, (-1, 1153))
    rst = rst.astype('uint8')

    return rst[:10856], rst[10856:]


#### 将转换的图片数组转为tfrecord文件

In [7]:
import tensorflow as tf


# 读取数据
data = images_convert()

# 转为tfrecord文件
config = ['train', 'validation']
for each in range(len(config)):
    mnist_type = config[each]
    # tfrecord格式文件名
    with tf.python_io.TFRecordWriter( mnist_type + '.tfrecords') as writer:
        image_path = data[each]
        for num_image in range(image_path.shape[0]):
            image = Image.fromarray(data[each][num_image][1:].reshape(48, 24))
            # plt.show()
            image = image.tobytes()
            label = data[each][num_image][0]
            example = tf.train.Example(features=tf.train.Features(feature={  
                    "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(label)])),  
                    'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image]))  
                }))
            writer.write(example.SerializeToString())